In [44]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from scipy.integrate import quad#
import mplhep as hep
import numpy as np
import pandas as pd
import sympy as sp
hep.style.use(hep.style.ATLAS)
from scipy.interpolate import interp1d
import matplotlib.ticker as ticker
mpl.rcParams['font.family'] = 'Times New Roman'
sns.set_theme(context='talk', style='white')

def scientific_formatter(x, pos):
    if x == 0:
        return r'$0$'  
    exponent = int(np.floor(np.log10(np.abs(x))))
    coeff = x / 10**exponent
    return r'${:.1f} \times 10^{{{}}}$'.format(coeff, exponent)

from sympy import latex
from sympy import init_printing
init_printing(use_latex=True)


In [45]:
DFdata_fo = pd.read_csv('freeze_out.csv')
DFdata_NA64 = pd.read_csv('/home/yoxara/Downloads/NA64 (1).csv')

#DFdata_NA64


#### Using euation 20(a) of the [ArXiv: 1610.06587](https://arxiv.org/abs/1610.06587)

In [46]:
delta_central, delta_min, delta_max = 251*10**(-11), (251-(2*51))*10**(-11), (251+(2*51))*10**(-11)  # Central, min, and max value of Delta a_mu
#delta_central, delta_min, delta_max = 105.47*10**(-11), (105.47-(2*61.92))*10**-11, (105.47+(2*61.92))*10**-11
 # Particle mass and parameters
m_mu = 0.105  # Muon mass in GeV 
epsilon = 1.0  # m_mu/m_mu
e  = np.sqrt(4*np.pi/137)
gs=DFdata_NA64['Epsilon_X']/e
#gs


In [47]:
#interpolation
interp_spline = interp1d(DFdata_NA64['m_phi'],gs, kind='linear', fill_value='extrapolate')
m_phi_extended = np.logspace(-3, np.log10(3), 100)
Epsilon_X_interpolated = interp_spline(m_phi_extended)


In [60]:

# New integrand function
def integrand_new(x, m_mu, m_S):
    numerator = m_mu**2 * (1 - x) * (1 - x**2)
    denominator = m_mu**2 * (1 - x)**2 + m_S**2 * x
    return numerator / denominator

m_S_values = np.linspace(0.1, 3, 1000)
gs_min_values_new = []
gs_max_values_new = []
gs_central_values_new = []

for m_S in m_S_values:
    integral_value_new, error = quad(integrand_new, 0, 1, args=(m_mu, m_S))
    gs_central_new = np.sqrt((delta_central * 8 * np.pi**2) / integral_value_new)  # if integral_value_new > 0 else np.nan
    gs_central_values_new.append(gs_central_new)
    gs_min_new = np.sqrt((delta_min * 8 * np.pi**2) / integral_value_new)  # if integral_value_new > 0 else np.nan
    gs_min_values_new.append(gs_min_new)
    gs_max_new = np.sqrt((delta_max * 8 * np.pi**2) / integral_value_new)  # if integral_value_new > 0 else np.nan
    gs_max_values_new.append(gs_max_new)

# Create DataFrame for new gs values
df_new = pd.DataFrame({
    'm_S': m_S_values, 
    'gs_min': gs_min_values_new, 
    'gs_max': gs_max_values_new,
    'gs_central': gs_central_values_new
}).astype(float)

# Plotting the new data
fig, ax = plt.subplots(figsize=(6,5))
#ax.plot(df_new['m_S'], df_new['gs_central'], linestyle='-', color='silver', linewidth=1)
ax.plot(df_new['m_S'], df_new['gs_min'], linestyle='-.', color='silver', linewidth=1)
ax.plot(df_new['m_S'], df_new['gs_max'], linestyle='-.', color='silver', linewidth=1)
ax.fill_between(df_new['m_S'], df_new['gs_min'], df_new['gs_max'], color="silver", alpha=0.3)
ax.text(0.2, 7.8e-4, r'$(g - 2)_\mu \pm 2\sigma$', fontsize=10, color='gray', rotation=5., ha='center', va='center')
ax.plot(DFdata_fo['m_phi'], DFdata_fo['g_mu'], linestyle='-', color='crimson', linewidth=2, label=r'freeze-out $\Omega_\chi$, only muon-interaction, $g_\chi=1, m_S=3 m_\chi$')
#ax.plot(DFdata_NA64['m_phi'], gs, linestyle='-', color='purple', linewidth=1.3)
plt.loglog(m_phi_extended, Epsilon_X_interpolated, linestyle='-', color='purple',  linewidth=2, label='NA64 (I nedd to check)') # label='Interpolated Curve'
ax.legend(loc='best', fontsize=10.5)
ax.set_xlabel(r'$m_{\phi}$ [GeV]', size=18, color='black')
ax.set_ylabel(r'$\lambda$', size=18, color='black')
# ax.yaxis.set_major_formatter(ticker.FuncFormatter(scientific_formatter))
ax.tick_params(direction='in', length=8, width=0.9, colors='black', which='major', top=True, right=True)
ax.tick_params(direction='in', length=3.2, width=0.7, colors='black', which='minor', top=True, right=True)
ax.xaxis.set_ticks_position('both')
ax.yaxis.set_ticks_position('both')
for spine in ax.spines.values():
    spine.set_linewidth(1.4)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylim([10**(-6), 3 * 10**(0)])
ax.set_xlim([10**(-1), 3 * 10**(0)])
ax.grid(True, which="both", ls="-", color='lightgray', alpha=0.008)
plt.tight_layout()
plt.show()
